# 3. Similitud entre productos
Descripción
Un desafío constante en MELI es el de poder agrupar productos similares utilizando
algunos atributos de estos como pueden ser el título, la descripción o su imagen.
Para este desafío tenemos un dataset “items_titles.csv” que tiene títulos de 30 mil
productos de 3 categorías diferentes de Mercado Libre Brasil

####  Objetivos : 
El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares son dos títulos del dataset “item_titles_test.csv” generando como output un listado, donde ordenando por score de similitud podamos encontrar los pares de productosmás similares en nuestro dataset de test.

Tarefas :
 1. Tokenizador de descritivos  [X]
 2. Método de similaridade  [X]
 3. Algoritmo eficiente para rodar em todo dataset [x]
 4. Gerar output [X]

In [22]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
dtitens = pd.read_csv('items_titles.csv')


In [23]:
# Duplicados
print('Itens Duplicados :')
dtitens[dtitens['ITE_ITEM_TITLE'].duplicated(keep='last')]



Itens Duplicados :


,ITE_ITEM_TITLE


In [24]:
print('Itens Nulos :')
dtitens[dtitens['ITE_ITEM_TITLE'].isnull()]

Itens Nulos :


,ITE_ITEM_TITLE


In [25]:
# Create similarity Function

In [26]:
import nltk
from nltk import tokenize  
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('portuguese')

def ehNaoStopW(palavra):
    return palavra not  in  stopwords

def getTokens(text):
    palavras_tokenize = tokenize.word_tokenize(text.lower(), language='portuguese') 
    filtered = filter(lambda token: ehNaoStopW(token) , palavras_tokenize)
    return list(filtered)



[nltk_data] Downloading package stopwords to /home/leo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1. uso do nltk para criar um vetor de tokens 
### 2. aplicar o filtro de stopwords é uma técnica clássica de NLP para evitar interferência de palavras de pouco significado semântico,  tais como  "a", "um" etc. 


In [27]:

def jaccard_similarity(x,y):
    """ returns the jaccard similarity between two lists """
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def similarity(text1,text2):
    return jaccard_similarity(getTokens(text1), getTokens(text2) )
    
    
text1 ="um bom dia para ir A Praia "
text2 ="um bom dia para surfar "
similarity(text1, text2)
    


0.4

### O índice de Jaccard, também conhecido como coeficiente de similaridade de Jaccard, é uma estatística usada para medir a similaridade e diversidade de conjuntos de amostras : 

![alt text](jacarta.jpeg "índice de Jaccard")

### Rodar o algoritmo por todo dataset  (1 tentativa )

In [28]:
%%time
def getsimilatitem(texto):
    maxSmilaridade =0 
    termo ="" 
    for i in range(dtitens.shape[0]):
        similaridade = similarity(text1, dtitens.iloc[i]['ITE_ITEM_TITLE'])
        if similaridade > maxSmilaridade : 
            maxSmilaridade = similaridade
            termo = dtitens.iloc[i]['ITE_ITEM_TITLE']
    return  termo , maxSmilaridade          

termo,maxSmilaridade = getsimilatitem("um bom dia para surfar")

CPU times: user 8.43 s, sys: 8.53 ms, total: 8.44 s
Wall time: 8.44 s


In [36]:
#  tempo execução em horas 
tempoexec = (17.5 * dtitens.shape[0]) / 3600
print("Metodo 1 para calcular {} linhas  leva {:.1f}  horas!! ".format(dtitens.shape[0],tempoexec))




Metodo 1 para calcular 30000 linhas  leva 145.8  horas!! 



### Rodar o algoritmo por todo dataset  (2 tentativa )

In [37]:
# cria base de dados tokenizada
textosBase =[]
for i in range(dtitens.shape[0]):
    texto = dtitens.iloc[i]['ITE_ITEM_TITLE']
    palavras_tokenize = getTokens(texto)
    textosBase.append( [texto,  palavras_tokenize] )                      
                            
                            

In [38]:
%%time
def getsimilatitem2(texto):
    maxSmilaridade =0 
    termo ="" 
    palavras_tokenize = getTokens(texto)
    for item in textosBase:
        if  item[0]!= texto:
            similaridade = jaccard_similarity(palavras_tokenize, item[1])
            if similaridade > maxSmilaridade : 
                maxSmilaridade = similaridade
                termo = item[0]
    return  termo , maxSmilaridade          

termo,maxSmilaridade = getsimilatitem2("um bom dia para surfar")
termo,maxSmilaridade

CPU times: user 67.4 ms, sys: 0 ns, total: 67.4 ms
Wall time: 66.5 ms


('Tênis Muito Bom', 0.25)

In [39]:
#  tempo execução em horas 
tempoexec = (0.195 * dtitens.shape[0]) / 3600
print("Método 2 para calcular {} linhas  leva {:.1f}  horas!! ".format(dtitens.shape[0],tempoexec))

Método 2 para calcular 30000 linhas  leva 1.6  horas!! 


In [93]:
# cria base de dados similaridade
from tqdm import tqdm

textosIndexados =[]
for i in tqdm(range(dtitens.shape[0])):
    texto = dtitens.iloc[i]['ITE_ITEM_TITLE']
    textosimilar,maxSmilaridade = getsimilatitem2(texto)
    textosIndexados.append((texto,textosimilar, maxSmilaridade) )
    

100%|██████████| 30000/30000 [44:17<00:00, 11.29it/s] 


In [96]:
df = pd.DataFrame (textosIndexados, columns = ['ITE_ITEM_TITLE','ITE_ITEM_TITLE', 'Score Similitud (0,1)' ])
df.to_csv("q2_similar_result.csv")
# dataset gerado : q2_similar_result.csv

In [97]:
df

,ITE_ITEM_TITLE,ITE_ITEM_TITLE,"Score Similitud (0,1)"
0,Tênis Ascension Posh Masculino - Preto E Verme...,Tênis Masculino Ascension Bx1949 - Preto E Mar...,0.555556
1,Tenis Para Caminhada Super Levinho Spider Corr...,Tenis Para Caminhada E Corrida,0.500000
2,Tênis Feminino Le Parc Hocks Black/ice Origina...,Tênis Feminino Hocks Skatista Le Parc Branco E...,0.545455
3,Tênis Olympikus Esportivo Academia Nova Tendên...,Tênis Olympikus Esportivo Academia Nova Tendên...,0.666667
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...,Sapatos De Menina Sapatos De Luz Led,0.222222
...,...,...,...
29995,Tênis Vans Old Skool I Love My Vans - Usado - ...,Tênis Old Skool Clássico Feminino,0.363636
29996,Tênis Feminino Preto Moleca 5296155,Tênis Feminino Moleca Preto Com Cadarço 5296155,0.833333
29997,Tenis Botinha Com Pelo Via Marte Original Lanç...,Botinha Com Pelo Tenis Via Marte Outono Invern...,0.625000
29998,Tênis Slip On Feminino Masculino Original Sapa...,Tênis Taylor Slip On Masculino Feminino Origin...,0.545455
